In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE182362"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE182362"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE182362.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE182362.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE182362.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"miR-155-regulated mTOR and Toll-like receptor 5 in gastric diffuse large B-cell lymphoma"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: B-cell lymphoma cell line U2932', 'tissue: cell line derived from diffuse large B-cell lymphoma'], 1: ['tissue: cell line derived from diffuse large B-cell lymphoma', 'treatment: transfected with miR-200c'], 2: ['treatment: transfected with an empty vector', 'treatment: transfected with miR-200a', 'treatment: transfected with miR-200b', nan]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import json
from typing import Dict, Any, Callable, Optional

# 1. Gene Expression Data Availability
# The "SuperSeries" description suggests this might include gene expression data
# alongside miRNA data, as it's common for SuperSeries to contain multiple data types
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics, we can determine:

# 2.1 Data Availability
# From the sample characteristics, all samples appear to be DLBCL cell lines,
# making the trait constant across all samples. As per instructions,
# constant features are considered not available.
trait_row = None  # Since all samples appear to be DLBCL

# Age and gender are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait data (DLBCL) to binary format."""
    if value is None:
        return None
    
    # Extract the value part after colon if it exists
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates DLBCL
    if 'diffuse large B-cell lymphoma' in value.lower():
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age data to numerical format."""
    # Not used in this dataset, but included for completeness
    return None

def convert_gender(value):
    """Convert gender data to binary format (0=female, 1=male)."""
    # Not used in this dataset, but included for completeness
    return None

# 3. Save Metadata
# Determine trait data availability based on trait_row
is_trait_available = trait_row is not None

# Save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We skip this step since trait_row is None (constant trait)
if trait_row is not None:
    # This block will be skipped based on our analysis
    # but keeping the structure in case trait_row changes in future
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # Use the clinical_data from previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:", preview)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"miR-155-regulated mTOR and Toll-like receptor 5 in gastric diffuse large B-cell lymphoma"
Line 1: !Series_geo_accession	"GSE182362"
Line 2: !Series_status	"Public on Feb 09 2022"
Line 3: !Series_submission_date	"Aug 18 2021"
Line 4: !Series_last_update_date	"Feb 09 2022"
Line 5: !Series_pubmed_id	"34913612"
Line 6: !Series_summary	"This SuperSeries is composed of the SubSeries listed below."
Line 7: !Series_overall_design	"Refer to individual Series"
Line 8: !Series_type	"Expression profiling by array"
Line 9: !Series_type	"Non-coding RNA profiling by array"
Found table marker at line 70
First few lines after marker:
"ID_REF"	"GSM5527571"	"GSM5527572"	"GSM5527573"	"GSM5527574"
"A_19_P00315452"	34.088	30.238	57.407	19.554
"A_19_P00315459"	903.948	986.916	922.612	764.945
"A_19_P00315469"	7.125	9.957	8.062	8.2
"A_19_P00315473"	6.314	24.339	8.542	7.055
Total lines examined: 71

Attempting to extract gene data from matrix file...
Suc

### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers in the gene expression data are in the format 'A_19_P00315452',
# which are Agilent microarray probe IDs rather than standard human gene symbols.
# These IDs need to be mapped to standard gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1259045 rows

Gene annotation preview (first few rows):
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}

Column names in gene annotation data:
['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the mapping columns
# The gene expression data has IDs in the format 'A_19_P00315452'
# In the gene annotation data, these IDs correspond to the 'ID' column
# The 'GENE_SYMBOL' column contains the gene symbols we want to map to

print("Preparing gene identifier mapping...")

# 2. Extract the mapping between probe IDs and gene symbols
# Get gene mapping dataframe by extracting the two relevant columns
try:
    # Extract ID and GENE_SYMBOL columns for mapping
    mapping_df = get_gene_mapping(gene_annotation, "ID", "GENE_SYMBOL")
    print(f"Created mapping dataframe with {len(mapping_df)} rows")
    
    # Check how many unique probes and genes are in the mapping
    unique_probes = mapping_df['ID'].nunique()
    unique_genes = mapping_df['Gene'].nunique()
    print(f"Mapping contains {unique_probes} unique probes and {unique_genes} unique genes")
    
    # 3. Convert probe-level measurements to gene expression data
    print("Converting probe measurements to gene expression data...")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Check the result
    if gene_data.empty:
        print("Warning: Gene expression dataframe is empty after mapping")
        is_gene_available = False
    else:
        print(f"Successfully mapped probe IDs to gene symbols. Final gene expression data has {len(gene_data)} genes.")
        print("First 10 gene symbols:")
        print(gene_data.index[:10])
        
        # Save the gene data to file
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data.to_csv(out_gene_data_file)
        print(f"Gene expression data saved to {out_gene_data_file}")
        
except Exception as e:
    print(f"Error in gene mapping process: {str(e)}")
    is_gene_available = False


Preparing gene identifier mapping...
Created mapping dataframe with 124298 rows
Mapping contains 56169 unique probes and 67260 unique genes
Converting probe measurements to gene expression data...


Successfully mapped probe IDs to gene symbols. Final gene expression data has 21773 genes.
First 10 gene symbols:
Index(['A-', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1', 'A1-'], dtype='object', name='Gene')
Gene expression data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE182362.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print("\nNormalizing gene symbols...")
# Get the gene data from previous step if needed
try:
    # First, check if we need to reload the gene data
    gene_data_path = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE182362.csv"
    if os.path.exists(gene_data_path):
        print("Loading gene data from previous step...")
        gene_data = pd.read_csv(gene_data_path, index_col=0)
    else:
        print("Gene data not found, recreating from previous steps...")
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        gene_annotation = get_gene_annotation(soft_file)
        gene_data = get_genetic_data(matrix_file)
        mapping_df = get_gene_mapping(gene_annotation, "ID", "GENE_SYMBOL")
        gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    print(f"Gene data shape before normalization: {gene_data.shape}")
    print("Sample of gene symbols before normalization:", gene_data.index[:10].tolist())
    
    # Use the normalize_gene_symbols_in_index function to standardize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} genes")
    
    # Handle case where normalization results in 0 genes
    if len(normalized_gene_data.index) == 0:
        print("WARNING: Normalization resulted in 0 genes. Using original gene data for diagnostics.")
        normalized_gene_data = gene_data  # Use original data for diagnostic purposes
        is_gene_available = False  # Mark that normalization failed
    else:
        is_gene_available = True
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Gene data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene normalization: {str(e)}")
    is_gene_available = False

# 2. Try to retrieve clinical data
print("\nChecking clinical data availability...")
try:
    # Recall from previous steps that trait_row was None, indicating no clinical data is available
    trait_row = None
    is_trait_available = trait_row is not None
    
    if is_trait_available:
        # This block should never execute as trait_row is None
        print("Clinical data is available.")
        if os.path.exists(out_clinical_data_file):
            clinical_df = pd.read_csv(out_clinical_data_file)
            print(f"Loaded clinical data with shape: {clinical_df.shape}")
        else:
            print("Clinical data file not found.")
            is_trait_available = False
    else:
        print("No clinical data available for this cohort. Cannot proceed with linking.")
except Exception as e:
    print(f"Error checking clinical data: {str(e)}")
    is_trait_available = False

# 3. Since clinical data is not available, we can't create linked data
linked_data = pd.DataFrame()  # Empty DataFrame as placeholder
is_biased = True  # Consider it biased since we can't evaluate properly

# 4. Validate and save cohort information
print("\nPerforming final validation...")
note = "Dataset does not contain clinical trait information for Large_B-cell_Lymphoma (all samples appear to be cell lines)."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 5. Report final status
print(f"\nDataset usability for {trait} association studies: {is_usable}")
print(f"Reason: {note}")


Normalizing gene symbols...
Loading gene data from previous step...
Gene data shape before normalization: (21773, 4)
Sample of gene symbols before normalization: ['A-', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1', 'A1-']
After normalization: 19577 genes
Gene data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE182362.csv

Checking clinical data availability...
No clinical data available for this cohort. Cannot proceed with linking.

Performing final validation...
Abnormality detected in the cohort: GSE182362. Preprocessing failed.

Dataset usability for Large_B-cell_Lymphoma association studies: False
Reason: Dataset does not contain clinical trait information for Large_B-cell_Lymphoma (all samples appear to be cell lines).
